In [3]:
from sklearn import model_selection

cv_split = model_selection.ShuffleSplit(n_splits = 10, test_size = .3, train_size = .6, random_state = 0 )
print(cv_split)
#选出10个样本，train 6个 test3个
for train_index, test_index in cv_split.split(np.ones((5,100))):#这个方法得到索引
    print(train_index)
    print(test_index)
#得出的是范围在5以内，index

ShuffleSplit(n_splits=10, random_state=0, test_size=0.3, train_size=0.6)
[1 3 4]
[2 0]
[1 4 3]
[0 2]
[4 0 2]
[1 3]
[2 4 0]
[3 1]
[3 1 0]
[4 2]
[4 1 0]
[2 3]
[1 3 0]
[2 4]
[4 2 0]
[3 1]
[2 1 3]
[0 4]
[3 1 0]
[4 2]


In [ ]:
train1_x, test1_x, train1_y, test1_y = model_selection.train_test_split(data1[data1_x_calc], data1[Target], random_state = 0)


model_selection.cross_validate(model, X, Y, cv  = cv_split)
cv取cv_split 我不懂他是fit了多少
cv可以取整数 如果取整数就是k-fold CV
将训练集划分为k个小的数据集，称之为k-fold CV。对每个fold进行下列过程：
* 1 用其他k-1 folds作为训练数据，训练模型
* 2 模型的结果用剩下的fold评价
模型的性能用上述循环中的k-fold交叉验证集的平均值表现。这种做法增加了计算量，但提高了数据的利用效率

In [ ]:
data1['Name'].str.split(',', expand=True) #得到DataFrame 如果没有expand，则还是Series，就是str改成了列表， 如果有expand就会变成DataFrame 增加column 0，1之类的

series.mode()  是出现次数最多的值 得到的是series
dataset['Embarked'].fillna(dataset['Embarked'].mode()[0], inplace = True) 填充nan

data1['Title'].apply(lambda x: 'Misc' if title_names.loc[x] == True else x)#series的每个值都应用func

model.__class__.__name__ model的名字

from sklearn.preprocessing import  LabelEncoder # 把sex等编码成0， 1
label = LabelEncoder()
dataset['Sex_Code'] = label.fit_transform(dataset['Sex'])

for x in data1_x:     #看每个
    print(data1[[x, Target[0]]].groupby(x, as_index=False).mean())


In [ ]:
#尝试所有模型  ， 然后把结果画出来
#Machine Learning Algorithm  (MLA) Selection and Initialization
MLA = [
    #Ensemble Methods
    ensemble.AdaBoostClassifier(),
    ensemble.BaggingClassifier(),
    ensemble.ExtraTreesClassifier(),
    ensemble.GradientBoostingClassifier(),
    ensemble.RandomForestClassifier(),

    #Gaussian Processes
    gaussian_process.GaussianProcessClassifier(),
    
    #GLM
    linear_model.LogisticRegressionCV(),
    linear_model.PassiveAggressiveClassifier(),
    linear_model.RidgeClassifierCV(),
    linear_model.SGDClassifier(),
    linear_model.Perceptron(),
    
    #Navies Bayes
    naive_bayes.BernoulliNB(),
    naive_bayes.GaussianNB(),
    
    #Nearest Neighbor
    neighbors.KNeighborsClassifier(),
    
    #SVM
    svm.SVC(probability=True),
    svm.NuSVC(probability=True),
    svm.LinearSVC(),
    
    #Trees    
    tree.DecisionTreeClassifier(),
    tree.ExtraTreeClassifier(),
    
    #Discriminant Analysis
    discriminant_analysis.LinearDiscriminantAnalysis(),
    discriminant_analysis.QuadraticDiscriminantAnalysis(),

    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
    XGBClassifier()    
    ]


#split dataset in cross-validation with this splitter class: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.ShuffleSplit.html#sklearn.model_selection.ShuffleSplit
#note: this is an alternative to train_test_split
cv_split = model_selection.ShuffleSplit(n_splits = 6, test_size = .3, train_size = .6, random_state = 0 ) # run model 10x with 60/30 split intentionally leaving out 10%

#create table to compare MLA metrics
MLA_columns = ['MLA Name', 'MLA Parameters','MLA Train Accuracy Mean', 'MLA Test Accuracy Mean', 'MLA Test Accuracy 3*STD' ,'MLA Time']
MLA_compare = pd.DataFrame(columns = MLA_columns) #创建空的DataFrame先把表头写出来

#create table to compare MLA predictions
MLA_predict = data1[Target]
print(type(MLA_predict))
print(type(MLA_predict['Survived']))  #上面Target是['Survived']所以得到的是DataFrame 直接str是Series
#index through MLA and save performance to table
row_index = 0
for alg in MLA:

    #set name and parameters
    MLA_name = alg.__class__.__name__
    MLA_compare.loc[row_index, 'MLA Name'] = MLA_name
    MLA_compare.loc[row_index, 'MLA Parameters'] = str(alg.get_params())
        #score model with cross validation: http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_validate.html#sklearn.model_selection.cross_validate

    cv_results = model_selection.cross_validate(alg, data1[data1_x_bin], data1[Target], cv=cv_split)
    print(cv_results)
    MLA_compare.loc[row_index, 'MLA Time'] = cv_results['fit_time'].mean()
    MLA_compare.loc[row_index, 'MLA Train Accuracy Mean'] = cv_results['train_score'].mean()
    MLA_compare.loc[row_index, 'MLA Test Accuracy Mean'] = cv_results['test_score'].mean()   
    #if this is a non-bias random sample, then +/-3 standard deviations (std) from the mean, should statistically capture 99.7% of the subsets
    MLA_compare.loc[row_index, 'MLA Test Accuracy 3*STD'] = cv_results['test_score'].std()*3   #let's know the worst that can happen!
    

    #save MLA predictions - see section 6 for usage
    alg.fit(data1[data1_x_bin], data1[Target])
    MLA_predict[MLA_name] = alg.predict(data1[data1_x_bin])
    
    row_index+=1
    
MLA_compare.sort_values(by = ['MLA Test Accuracy Mean'], ascending = False, inplace = True)


In [ ]:
sns.barplot(x='MLA Test Accuracy Mean', y = 'MLA Name', data = MLA_compare, color = 'm')

#prettify using pyplot: https://matplotlib.org/api/pyplot_api.html
plt.title('Machine Learning Algorithm Accuracy Score \n')
plt.xlabel('Accuracy Score (%)')
plt.ylabel('Algorithm')

In [ ]:
#超参数调试
#先建立超参数坐标 字典
param_grid = {'criterion': ['gini', 'entropy'],  #scoring methodology; two supported formulas for calculating information gain - default is gini
              #'splitter': ['best', 'random'], #splitting methodology; two supported strategies - default is best
              'max_depth': [2,4,6,8,10,None], #max depth tree can grow; default is none
              #'min_samples_split': [2,5,10,.03,.05], #minimum subset size BEFORE new split (fraction is % of total); default is 2
              #'min_samples_leaf': [1,5,10,.03,.05], #minimum subset size AFTER new split split (fraction is % of total); default is 1
              #'max_features': [None, 'auto'], #max features to consider when performing split; default none or all
              'random_state': [0] #seed or control random number generator: https://www.quora.com/What-is-seed-in-random-number-generation
             }

tune_model = model_selection.GridSearchCV(tree.DecisionTreeClassifier(), param_grid=param_grid, scoring = 'roc_auc', cv = cv_split)
tune_model.fit(data1[data1_x_bin], data1[Target])
tune_model.cv_results_  #是所有结果 （字典）
tune_model.best_params_  #最好的参数

tune_model.cv_results_['mean_train_score'][tune_model.best_index_]*100
tune_model.cv_results_['mean_train_score'] #得到是array  后面得到的是最好split中的一个的索引 得到一个分数

In [1]:
#特征选择
from sklearn import feature_selection
dtree_rfe = feature_selection.RFECV(dtree, step = 1, scoring = 'accuracy', cv = cv_split)
dtree_rfe.fit(data1[data1_x_bin], data1[Target])

#dtree_rfe.get_support() 得到列表  array类型的布尔值
X_rfe = data1[data1_x_bin].columns.values[dtree_rfe.get_support()] #可以得到选择后的特征列表
rfe_results = model_selection.cross_validate(dtree, data1[X_rfe], data1[Target], cv  = cv_split)#用选择后的特征试

#也可以用特征选择+超参数搜素

NameError: name 'dtree' is not defined

In [ ]:
#使用投票分类器
#硬分类是谁大就是谁，软分类是按概率
vote_est = [
    #Ensemble Methods: http://scikit-learn.org/stable/modules/ensemble.html
    ('ada', ensemble.AdaBoostClassifier()),
    ('bc', ensemble.BaggingClassifier()),
    ('etc',ensemble.ExtraTreesClassifier()),
    ('gbc', ensemble.GradientBoostingClassifier()),
    ('rfc', ensemble.RandomForestClassifier()),

    #Gaussian Processes: http://scikit-learn.org/stable/modules/gaussian_process.html#gaussian-process-classification-gpc
    ('gpc', gaussian_process.GaussianProcessClassifier()),
    
    #GLM: http://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
    ('lr', linear_model.LogisticRegressionCV()),
    
    #Navies Bayes: http://scikit-learn.org/stable/modules/naive_bayes.html
    ('bnb', naive_bayes.BernoulliNB()),
    ('gnb', naive_bayes.GaussianNB()),
    
    #Nearest Neighbor: http://scikit-learn.org/stable/modules/neighbors.html
    ('knn', neighbors.KNeighborsClassifier()),
    
    #SVM: http://scikit-learn.org/stable/modules/svm.html
    ('svc', svm.SVC(probability=True)),
    
    #xgboost: http://xgboost.readthedocs.io/en/latest/model.html
   ('xgb', XGBClassifier())

]


#Hard Vote or majority rules
vote_hard = ensemble.VotingClassifier(estimators = vote_est , voting = 'hard')
vote_hard_cv = model_selection.cross_validate(vote_hard, data1[data1_x_bin], data1[Target], cv  = cv_split)
vote_hard.fit(data1[data1_x_bin], data1[Target])

In [ ]:
#scipt后面还写了for循环调试所有模型的超参数，然后再 用投票分类

In [ ]:
#tensorflow看网络正确率的方法
predict = (y_ > 0.5)
correct_prediction = tf.equal(predict, (y > 0.5))
accarcy = tf.reduce_mean(tf.cast(correct_prediction, 'float'))

#{}通用格式字符
print('test score : {:.2f}%'.format(sess.run(accarcy, feed_dict={x: split_test[:, 1:], y: split_test[:, 0].reshape(-1, 1)})*100.0))

In [ ]:
df1=pd.DataFrame({'key':['a','b','d'],'data1':range(3)})  
df2=pd.DataFrame({'key':['a','b','c'],'data2':range(3)})
pd.merge(df1,df2, how='outer')
#主要用于两个dataframe里面有一个column name是相同的，会得到len(columns(df1))+len(columns(df2))-1的column长度
#how 有4个 默认是inner 是吧两个key做交集，outer是并集， left是按照左边的，right是右边
#如果没有共同列 用on = [key1, key2]，类似pd.concat，

In [ ]:
######################过滤############################

train_df = df.filter(regex='Survived|Age_.*|SibSp|Parch|Fare_.*|Cabin_.*|Embarked_.*|Sex_.*|Pclass_.*')

In [ ]:
##############下面的代码是从transfrom恢复回来的尝试

In [1]:
a = np.ones((3,3))
a[1] = 0
b = np.ones((3,3))

NameError: name 'np' is not defined

In [ ]:
a

In [2]:
scaler_temp = preprocessing.StandardScaler()
scaler_temp.fit(a)
print(scaler_temp.mean_)
# print(scaler_temp.)

NameError: name 'preprocessing' is not defined

In [ ]:
scaler_temp2 = preprocessing.StandardScaler()
scaler_temp2.fit(b)
scaler_temp2.fit(a)
print(scaler_temp2.mean_)
scaler_temp2.var_

In [ ]:
((1-0.666)**2 + (0.666)**2 + (1-0.666)**2)/3

In [ ]:
c = scaler_temp.fit_transform(a)

In [ ]:
c

In [ ]:
c.mean(axis=0)

In [ ]:
c.var(axis=0)

In [ ]:
scaler_temp.fit_transform(X = a[:, :2], y = a[:, 2])

In [ ]:
(1-0.666)/np.sqrt(0.2222)

In [ ]:
-0.666/np.sqrt(0.2222)

In [ ]:
c

In [ ]:
scaler_temp.mean_

In [ ]:
scaler_temp.var_

In [ ]:
d = np.sqrt(scaler_temp.var_[1])
c[:, 2]*d + scaler_temp.mean_[1]

In [ ]:
c[:, 2] = c[:, 2]*d + scaler_temp.mean_[1]

In [ ]:
c

In [ ]:
train_df.pop('SalePrice')  #return 这列数据，并在原来的df中删除这列

In [ ]:
all_df = pd.concat((train_df, test_df), axis=0)  #另外一种train和test数据一起预处理的方式

In [ ]:
all_dummy_df.isnull().sum().sort_values(ascending=False).head(10)  #看缺失值

In [1]:
import numpy as np
np.logspace(1,2, 10)  #power(base, x)  默认base是10 可以改 endpoint就是包括不包括最后的点

array([  10.        ,   12.91549665,   16.68100537,   21.5443469 ,
         27.82559402,   35.93813664,   46.41588834,   59.94842503,
         77.42636827,  100.        ])

In [ ]:
train_df = pd.read_csv('../input/train.csv', index_col=0) #这样读进来没有索引 ， 没有索引才可以用后面的方式恢复 concat到一起的all_df
test_df = pd.read_csv('../input/test.csv', index_col=0)

train_df, test_df = all_dummies.loc[train_df.index], all_dummies.loc[test_df.index]

In [5]:
from sklearn.metrics import regression

In [11]:
regression.mean_squared_error([1,2], [10,1])

41.0

In [10]:
81/2

40.5

In [12]:
import xgboost as xgb


In [13]:
aa = xgb.XGBRegressor()

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [ ]:
model_selection.GridSearchCV()

In [ ]:
data1['Name'].str.split(', ', expand=True)[1].str.split('.', expand=True)[0].value_counts() #expand=True变成DataFrame自动添加类似0,1column 

In [ ]:
#随机排列数据的方法   
#产生一个随机排列的 序号  再选取

permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))

    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k * mini_batch_size : (k+1) * mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k * mini_batch_size : (k+1) * mini_batch_size]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)

        #最后一个batch不是整数
    if m % mini_batch_size != 0:
        mini_batch_X = shuffled_X[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch_Y = shuffled_Y[:, num_complete_minibatches * mini_batch_size : m]
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)


In [ ]:
# Calculate the correct predictions  
    correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))  
    # Calculate accuracy on the test set  
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))  